In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch.cuda.amp import autocast
import time
from torch import nn
from torch.nn.functional import dropout
from tqdm import tqdm
import sys
import glob
from collections import OrderedDict
import cv2
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
from albumentations import Compose, Normalize, Resize
from albumentations.pytorch import ToTensor, ToTensorV2

In [ ]:
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
import timm
from timm.models.layers.adaptive_avgmax_pool import SelectAdaptivePool2d
from timm.models.resnet import Bottleneck

In [ ]:
testdir = "../input/ranzcr-clip-catheter-line-classification/test"
df = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")
public_df = pd.read_csv("../input/ranzcrsubmissionfiles/PublicLB_A.csv").iloc[50:]

run_df = df[~df["StudyInstanceUID"].isin(public_df["StudyInstanceUID"].values)]

In [ ]:
class InverseTestAugment():
    def __init__(self):
        transformation = [
            Resize(256, 256),
            Normalize(),
            ToTensorV2()
            ]

        self.transform = Compose(transformation)

    def __call__(self, image):
        transformed = self.transform(image=image)
        return transformed['image']

class ClassificationDataset(Dataset):
    def __init__(self, dir_, names):
        self.dir = dir_
        self.names = names
        self.transform = InverseTestAugment()
    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        image = cv2.imread(os.path.join(self.dir, self.names[idx] + ".jpg"))
        image = self.transform(image=image)
        return image, self.names[idx] 

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model("tf_efficientnet_b0_ns", pretrained=False)
        
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 5)

    def forward(self, x):
        with autocast():
            bs = x.size(0)
            features = self.model(x)
            pooled_features = self.pooling(features).view(bs, -1)
            output = self.fc(pooled_features)
            return output

In [ ]:
model = CustomModel().cuda()
model.load_state_dict(torch.load("../input/ranzcr-images-error/inverse_model.pth", "cpu"))
model.eval()

In [ ]:
test_dataloader = DataLoader(ClassificationDataset("../input/ranzcr-clip-catheter-line-classification/test", run_df["StudyInstanceUID"].values), batch_size=16, pin_memory=True, shuffle=False, num_workers=4)

In [ ]:
tbar = tqdm(test_dataloader)
preds = []
names = []
with torch.no_grad():
    for image, name in tbar:
        image = image.cuda()
        output = model(image)            
        preds.append(output.cpu().numpy())
        names.append(name)
    preds = torch.argmax(torch.tensor(np.vstack(preds)), 1).numpy()
    names = np.hstack(names)

In [ ]:
inverse_dict = {}
for n,p in zip(names, preds):
    inverse_dict[n] = p

In [ ]:
wfiles = sorted(glob.glob("../input/clip-weights/*.pth"))

In [ ]:
wfiles

In [ ]:
wfiles = [
 '../input/clip-weights/b6rf_fold0.pth',
 '../input/clip-weights/b6rf_fold1.pth',
 '../input/clip-weights/b6rf_fold2.pth',
 '../input/clip-weights/b6rf_fold3.pth',
 '../input/clip-weights/b6rf_fold4.pth',
 '../input/clip-weights/b7_fold0_1024_97188.pth',
 '../input/clip-weights/b7_fold0_1024_97363.pth',
 '../input/clip-weights/b7_fold1_1024_97092.pth',
 '../input/clip-weights/b7_fold1_1024_97213.pth',
 '../input/clip-weights/b7_fold2_1024_97135.pth',
 '../input/clip-weights/b7_fold2_1024_97165.pth',
 '../input/clip-weights/b7_fold3_1024_97326.pth',
 '../input/clip-weights/b7_fold3_1024_97388.pth',
 '../input/clip-weights/b7_fold4_1024_97500.pth',
 '../input/clip-weights/b7_fold4_1024_97587.pth']

In [ ]:
for i, wf in enumerate(wfiles):
    pt = OrderedDict()
    ckpt = torch.load(wf, "cpu").pop("state_dict")
    for k,v in ckpt.items():
        pt[k[7:]] = v
    wfiles[i] = pt

In [ ]:
class EfficientNet(nn.Module):
    """
    EfficientNet B0-B8.
    Args:
        cfg (CfgNode): configs
    """
    def __init__(self, model_name):
        super(EfficientNet, self).__init__()
        backbone = timm.create_model(
            model_name=model_name,
            pretrained=False,
            in_chans=3,
        )
        self.conv_stem = backbone.conv_stem
        self.bn1 = backbone.bn1
        self.act1 = backbone.act1
        ### Original blocks ###
        for i in range(len((backbone.blocks))):
            setattr(self, "block{}".format(str(i)), backbone.blocks[i])
        self.conv_head = backbone.conv_head
        self.bn2 = backbone.bn2
        self.act2 = backbone.act2
        self.global_pool = SelectAdaptivePool2d(pool_type="avg")
        self.num_features = backbone.num_features
        self.bottleneck_b4 = Bottleneck(inplanes=self.block4[-1].bn3.num_features,
                                        planes=int(self.block4[-1].bn3.num_features / 4))
        self.bottleneck_b5 = Bottleneck(inplanes=self.block5[-1].bn3.num_features,
                                        planes=int(self.block5[-1].bn3.num_features / 4))
        self.fc_b4 = nn.Linear(self.block4[-1].bn3.num_features, 14)
        self.fc_b5 = nn.Linear(self.block5[-1].bn3.num_features, 14)

        self.fc = nn.Linear(self.num_features, 14)
        del backbone

    def _features(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x); b4 = x
        x = self.block5(x); b5 = x
        x = self.block6(x)
        x = self.conv_head(x)
        x = self.bn2(x)
        x = self.act2(x)
        return b4,b5,x

    def forward(self, x):
        with autocast():
            b4, b5, x = self._features(x)
            x = self.global_pool(x)
#             b4_logits = self.fc_b4(torch.flatten(self.global_pool(self.bottleneck_b4(b4)), 1))
            b5_logits = self.fc_b5(torch.flatten(self.global_pool(self.bottleneck_b5(b5)), 1))
            x = torch.flatten(x, 1)
            logits = self.fc(x)
            output = (torch.sigmoid(logits) + torch.sigmoid(b5_logits)) / 2.
            return output

In [ ]:
for i in range(5):
    exec(f"modelb6_{i} = EfficientNet(\"tf_efficientnet_b6\").cuda()")
    exec(f"modelb7a_{i} = EfficientNet(\"tf_efficientnet_b7\").cuda()")
    exec(f"modelb7b_{i} = EfficientNet(\"tf_efficientnet_b7\").cuda()")

In [ ]:
models = []
for m in ["b6", "b7a", "b7b"]:
    for i in range(5):
        exec(f"models.append(model{m}_{i})")

In [ ]:
for i in range(len(models)):
    models[i].load_state_dict(wfiles[i])
    models[i].eval()

In [ ]:
class to_tensor:
    def __init__(self, size):
        transformation = [
            Resize(size, size),                         
            Normalize(),
            ToTensor()]
        self.transform = Compose(transformation)

    def __call__(self, x):
        return self.transform(image=x)['image']

class CLiP(Dataset):
    def __init__(self, df, testdir):
        self.names = df["StudyInstanceUID"].values
        self.testdir = testdir
        self.to_tensor_1024 = to_tensor(1024)
        self.to_tensor_1344 = to_tensor(1344)
        
    def __getitem__(self, idx):
        imgpath = os.path.join(self.testdir, self.names[idx] + ".jpg")
        imgstate = inverse_dict[self.names[idx]]            
        imgraw = cv2.imread(imgpath)
        if imgstate == 1:
            imgraw = cv2.bitwise_not(imgraw)
        elif imgstate == 4:
            imgraw = imgraw[::-1,:,:]
        img1024 = self.to_tensor_1024(cv2.resize(imgraw, (1024, 1024)))
        img1344 = self.to_tensor_1344(cv2.resize(imgraw, (1504, 1504)))
        
        return img1024, img1344, self.names[idx]
    
    def __len__(self):
        return len(self.names)

In [ ]:
dataloader = DataLoader(CLiP(run_df, testdir), 8, shuffle=False, drop_last=False, num_workers=4)

In [ ]:
outputs = []
names = []
for i, (img1024, img1344, name) in tqdm(enumerate(dataloader)):
    img1024 = img1024.cuda()
    img1344 = img1344.cuda()
    output = 0
    with torch.no_grad():
        
        output += 0.3 * modelb7a_0(img1024)
        output += 0.3 * modelb7a_1(img1024)
        output += 0.3 * modelb7a_2(img1024)
        output += 0.3 * modelb7a_3(img1024)
        output += 0.3 * modelb7a_4(img1024)
        
        output += 0.3 * modelb7b_0(img1024)
        output += 0.3 * modelb7b_1(img1024)
        output += 0.3 * modelb7b_2(img1024)
        output += 0.3 * modelb7b_3(img1024)
        output += 0.3 * modelb7b_4(img1024)
        
        output += 0.4 * modelb6_0(img1344)
        output += 0.4 * modelb6_1(img1344)
        output += 0.4 * modelb6_2(img1344)
        output += 0.4 * modelb6_3(img1344)
        output += 0.4 * modelb6_4(img1344)
    outputs.append(output.cpu().numpy())
    names.append(name)

In [ ]:
data = []
if len(outputs) and len(names):
    outputs = np.vstack(outputs)
    names = np.hstack(names)
    for output, name in zip(outputs, names):
        data.append([name] + list(output[:-3] / 5.))
submission = pd.DataFrame(data, columns=df.columns)

In [ ]:
submission.head()

In [ ]:
submission = pd.concat([public_df, submission])

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)